In [1]:
%matplotlib qt
import numpy as np
import lmfit
from bluesky import RunEngine
from ophyd import Device, Signal
from ophyd import Component as Cpt
from ophyd.sim import det, motor1, motor2, noisy_det, SynGauss, SynAxis, SynSignal,EnumSignal
from bluesky.callbacks.best_effort import BestEffortCallback
from bluesky.callbacks import LiveFitPlot, LiveFit, LivePlot
from bluesky.callbacks.fitting import PeakStats
from bluesky.callbacks.mpl_plotting import plot_peak_stats
from bluesky.plans import scan, rel_scan, list_scan, grid_scan
from bluesky.plan_stubs import mv
from databroker import Broker
from matplotlib.pyplot import ion, subplots
from bluesky.utils import install_nb_kicker
from scipy.special import erf
install_nb_kicker()
ion()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""
/opt/micromamba/envs/bluesky-workshop/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


## RunEngine

In [2]:
RE = RunEngine({})
bec = BestEffortCallback()
RE.subscribe(bec)

0

## Device simulado

##### É possível criar devices que simulam o comportamento de algum detector ou fenômeno específico, nesse caso há um modelo de um detector que realiza a integral de um sinal com perfil gaussiano. Na prática, qualquer tipo de device pode ser implementado para simular experimentos ou scans na linha de luz.

In [3]:
import numpy as np #Hide this import here just for now

def gaussian_integral(x, peak, sigma, center):
    return peak * sigma * np.sqrt(np.pi / 2) * (erf((x - center) / (np.sqrt(2) * sigma)) - erf((-5 - center) / (np.sqrt(2) * sigma)))

class SynKnifeDetector(SynGauss):
    
    def __init__(self, name, motor, motor_field, center, Imax, *, random_state=None, **kwargs):
        super().__init__(name, motor, motor_field, center, Imax, **kwargs)

    def _compute(self):
        m = self._motor.read()[self._motor_field]["value"]
        Imax = self.Imax.get()
        center = self.center.get()
        sigma = self.sigma.get()
        noise = self.noise.get()
        noise_multiplier = self.noise_multiplier.get()
        
        return gaussian_integral(m, Imax, sigma, center)
        


## Instância dos devices simulados

#### É possível definir os parâmetros na inicialização do device **detector**

In [4]:
motor = SynAxis(name='motor')
motor.delay = 0.15
detector = SynKnifeDetector('detector', motor, 'motor', center=2, Imax=9, sigma=1)

### Plot fitting


#### Nesse caso, podemos testar funções como o LiveFitPlot e obter o resultado do fit ao final da run

In [5]:
fig, ax = subplots()
bec.disable_plots()

    
model = lmfit.Model(gaussian_integral)

init_guess = {'peak': 5, 'sigma': 1.5, 'center': 3}


live_fit = LiveFit(model, 'detector', {'x': 'motor'}, init_guess)
live_fit_plot = LiveFitPlot(live_fit, color='r', ax=ax, label='Fit')
live_plot = LivePlot('detector', 'motor', marker='x', linestyle='none', ax=ax, label='Scan')

RE(
    scan([detector], motor, -10, 10, num=40),
    [live_fit_plot, live_plot]
)



Transient Scan ID: 1     Time: 2024-09-03 10:13:41
Persistent Unique Scan ID: '269116f2-2465-452d-9283-4c96c571851f'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |      motor |   detector |
+-----------+------------+------------+------------+
|         1 | 10:13:42.1 |    -10.000 |     -0.000 |
|         2 | 10:13:42.3 |     -9.487 |     -0.000 |
|         3 | 10:13:42.4 |     -8.974 |     -0.000 |
|         4 | 10:13:42.6 |     -8.462 |     -0.000 |
|         5 | 10:13:42.8 |     -7.949 |     -0.000 |
|         6 | 10:13:42.9 |     -7.436 |     -0.000 |
|         7 | 10:13:43.1 |     -6.923 |     -0.000 |
|         8 | 10:13:43.3 |     -6.410 |     -0.000 |
|         9 | 10:13:43.5 |     -5.897 |     -0.000 |
|        10 | 10:13:43.6 |     -5.385 |     -0.000 |
|        11 | 10:13:43.8 |     -4.872 |      0.000 |
|        12 | 10:13:44.0 |     -4.359 |      0.000 |
|        13 | 10:13:44.1 |     -3.846 |      0.000 |
|        14

('269116f2-2465-452d-9283-4c96c571851f',)

In [6]:
live_fit.result.

In [7]:
data = db[-1].table() #Should we use databroker now?

NameError: name 'db' is not defined